In [ ]:
# How to use a HuggingFace dataset with fastai?
- toc: false 
- badges: true
- categories: [fastai, huggingface, datasets]
- comments: true

In this tutorial, we will show you how to use a HuggingFace dataset with fastai to train a model for image classification. We will use the [Beans dataset](https://huggingface.co/datasets/beans), which consists of images of beans with three different types of diseases.

## Step 1: Install the required libraries
Before starting, we need to install the required libraries. Run the following commands to install fastai and HuggingFace's datasets:

In [ ]:
!pip install -Uqq fastai
!pip install -Uqq datasets

Login to HuggingFace Hub to download the dataset.

In [ ]:
!huggingface-cli login

## Step 2: Import the required modules



In [ ]:
import torch
from torchvision import transforms
from fastai.data.all import *
from fastai.vision.all import *
from datasets import load_dataset


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

## Step 3: Load the dataset


Let's load [Beans dataset](https://huggingface.co/datasets/beans), which is a dataset of images of beans taken in the field using smartphone cameras. It consists of 3 classes: 2 disease classes and the healthy class.

In [ ]:
raw_ds = load_dataset("beans")
raw_ds


The dataset is splitted into train, validation, and test sets.

Let's see label names.

In [ ]:
class_names = raw_ds['train'].features['labels'].names
class_names

## Step 4: Preprocess the dataset

Often, we need preprocessing raw data before training a model with it. HuggingFace `datasets` library provides two methods for preprocessing:

1. `map`: This method is used to apply a function to each example in the dataset, possibly in a batched manner. The function can be applied to one or more columns of the dataset, and the result can be stored in a new column or overwrite the existing one. The map function also allows you to remove some columns from the dataset, if needed. This method is useful for preprocessing the dataset, such as resizing images, tokenizing text, or encoding categorical features. It caches outputs so that they're not computed again.

2. `set_transforms`: This method is used to set a transform function that is applied on-the-fly when accessing examples from the dataset. This means that the dataset is not modified in-place, and the transform function is applied only when the examples are accessed. This method is useful for applying data augmentation techniques or normalization that should be applied dynamically during training without modifying the dataset beforehand.



Let's resize each image to 224x244.

In [ ]:
def preprocess(records):
    records["image"] = [image.convert("RGB").resize((224, 224)) for image in records["image"]]
    return records

Before batching samples, we need to remove unnecessary columns in the dataset such as `image_file_path`.

In [ ]:
ds = raw_ds.map(preprocess, remove_columns=["image_file_path"], batched=True)

In [ ]:
ds['train'][0]

We won't use `set_transform` as fastai's DataBlock can apply item-level and batch-level transforms, e.g. data augmentations. When we use a pretrained model, it actually applies necessary transforms such as normalization automatically.

## Step 5: Create the DataBlock

Now, we can create dataloaders for the dataset using DataBlock from fastai. As the dataset is already splitted into train, validation, and test sets, we don't need to split it further. Hence, we will use `nosplit` function.

In [ ]:
def nosplit(items): 
    return list(range(len(items))), []

In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=lambda record: record['image'],
    get_y=lambda record: class_names[record['labels']],
    splitter = nosplit,
)

train_dl = dblock.dataloaders(ds['train']).train
valid_dl = dblock.dataloaders(ds['validation']).train
dls = DataLoaders(train_dl, valid_dl, device=DEVICE)
dls.show_batch()

In [ ]:
len(dls.train.dataset),len(dls.valid.dataset)

## Step 6: Training

Let's fine-tune a pretrained ResNet model on our dataset using `Learner.fine_tune` method.


In [ ]:
# Create a learner
learn = vision_learner(
    dls, 
    resnet34, 
    loss_func=CrossEntropyLossFlat(),
    metrics=accuracy, 
)

In [ ]:
# Find a good learning rate
learn.lr_find()

In [ ]:
# Fine-tune the model
learn.fine_tune(3, 1e-3, freeze_epochs=1)

## Step 7: Evaluation

Fine-tuned model achieves 93.97% accuracy on validation set, not too bad!

In [ ]:
loss, accuracy = learn.validate(dl=dls.valid)
print(f"Loss {loss:.6f}\nAccuracy: {accuracy:.2%}")

Let's check predictions visually.

In [ ]:
learn.show_results(dl=valid_dl)

Let's predict and evaluate on test set.

In [ ]:
tst_dl = dls.test_dl(ds['test'], with_labels=True)
probs, targets, preds = learn.get_preds(dl=tst_dl, with_decoded=True)

In [ ]:
loss, accuracy = learn.validate(dl=tst_dl)
print(f"Loss {loss:.6f}\nAccuracy: {accuracy:.2%}")